In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [18]:
data_test = pd.read_csv('/content/sample_data/Test.csv')
data_train = pd.read_csv('/content/sample_data/Train.csv')

In [19]:
data_train

,pickup_datetime,dropoff_datetime,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,passenger_count,traffic_congestion_level,pickup_borough,dropoff_borough,tolls_amount,tip_amount,fare_estimate,trip_distance_miles
0,3/22/24 9:10,3/22/24 9:23,40.690045,-74.239699,40.704018,-74.214368,2,high,Other,Other,0.00,0.14,18.40,10.9422
1,8/14/24 10:48,8/14/24 10:52,40.763224,-74.053604,40.759277,-74.066002,1,medium,Other,Other,0.00,1.36,9.99,1.2817
2,6/17/24 3:47,6/17/24 3:55,40.658970,-74.179122,40.628631,-74.180385,1,low,Other,Staten Island,5.67,2.36,20.22,13.8034
3,6/25/24 0:45,6/25/24 1:04,40.696980,-73.773970,40.670444,-73.838194,2,medium,Brooklyn,Brooklyn,0.00,3.80,29.73,24.2266
4,6/10/24 4:10,6/10/24 4:14,40.580638,-73.788101,40.594286,-73.780625,3,low,Brooklyn,Brooklyn,0.00,2.68,11.05,3.4222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14694,6/21/24 9:42,6/21/24 9:53,40.818300,-74.037218,40.823860,-74.005730,4,medium,Bronx,Bronx,0.00,1.19,14.59,9.4093
14695,9/21/24 21:39,9/21/24 21:56,40.913272,-74.012950,40.936902,-74.031666,1,high,Bronx,Bronx,0.00,3.58,20.40,12.7578
14696,1/16/24 23:48,1/16/24 23:57,40.808487,-74.072608,40.804565,-74.033420,2,medium,Bronx,Bronx,0.00,2.69,14.64,12.3564
14697,9/28/24 9:34,9/28/24 10:27,40.605883,-73.902029,40.693266,-73.843018,3,high,Brooklyn,Brooklyn,0.00,13.61,60.69,27.0350


In [20]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14699 entries, 0 to 14698
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   pickup_datetime           14699 non-null  object 
 1   dropoff_datetime          14699 non-null  object 
 2   pickup_latitude           14699 non-null  float64
 3   pickup_longitude          14699 non-null  float64
 4   dropoff_latitude          14699 non-null  float64
 5   dropoff_longitude         14699 non-null  float64
 6   passenger_count           14699 non-null  int64  
 7   traffic_congestion_level  14699 non-null  object 
 8   pickup_borough            14699 non-null  object 
 9   dropoff_borough           14699 non-null  object 
 10  tolls_amount              14699 non-null  float64
 11  tip_amount                14699 non-null  float64
 12  fare_estimate             14699 non-null  float64
 13  trip_distance_miles       14699 non-null  float64
dtypes: flo

In [21]:
data_train.isna().sum().sort_values(ascending=False)

,0
pickup_datetime,0
dropoff_datetime,0
pickup_latitude,0
pickup_longitude,0
dropoff_latitude,0
dropoff_longitude,0
passenger_count,0
traffic_congestion_level,0
pickup_borough,0
dropoff_borough,0


In [22]:
data_train['lat_diff'] = data_train['dropoff_latitude'] - data_train['pickup_latitude']
data_train['lon_diff'] = data_train['dropoff_longitude'] - data_train['pickup_longitude']
data_train['euclidean_distance'] = np.sqrt(data_train['lat_diff']**2 + data_train['lon_diff']**2)

In [23]:
data_train

,pickup_datetime,dropoff_datetime,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,passenger_count,traffic_congestion_level,pickup_borough,dropoff_borough,tolls_amount,tip_amount,fare_estimate,trip_distance_miles,lat_diff,lon_diff,euclidean_distance
0,3/22/24 9:10,3/22/24 9:23,40.690045,-74.239699,40.704018,-74.214368,2,high,Other,Other,0.00,0.14,18.40,10.9422,0.013973,0.025331,0.028929
1,8/14/24 10:48,8/14/24 10:52,40.763224,-74.053604,40.759277,-74.066002,1,medium,Other,Other,0.00,1.36,9.99,1.2817,-0.003947,-0.012398,0.013011
2,6/17/24 3:47,6/17/24 3:55,40.658970,-74.179122,40.628631,-74.180385,1,low,Other,Staten Island,5.67,2.36,20.22,13.8034,-0.030339,-0.001263,0.030365
3,6/25/24 0:45,6/25/24 1:04,40.696980,-73.773970,40.670444,-73.838194,2,medium,Brooklyn,Brooklyn,0.00,3.80,29.73,24.2266,-0.026536,-0.064224,0.069490
4,6/10/24 4:10,6/10/24 4:14,40.580638,-73.788101,40.594286,-73.780625,3,low,Brooklyn,Brooklyn,0.00,2.68,11.05,3.4222,0.013648,0.007476,0.015561
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14694,6/21/24 9:42,6/21/24 9:53,40.818300,-74.037218,40.823860,-74.005730,4,medium,Bronx,Bronx,0.00,1.19,14.59,9.4093,0.005560,0.031488,0.031975
14695,9/21/24 21:39,9/21/24 21:56,40.913272,-74.012950,40.936902,-74.031666,1,high,Bronx,Bronx,0.00,3.58,20.40,12.7578,0.023630,-0.018716,0.030144
14696,1/16/24 23:48,1/16/24 23:57,40.808487,-74.072608,40.804565,-74.033420,2,medium,Bronx,Bronx,0.00,2.69,14.64,12.3564,-0.003922,0.039188,0.039384
14697,9/28/24 9:34,9/28/24 10:27,40.605883,-73.902029,40.693266,-73.843018,3,high,Brooklyn,Brooklyn,0.00,13.61,60.69,27.0350,0.087383,0.059011,0.105442


In [24]:
data_train['pickup_datetime'] = pd.to_datetime(data_train['pickup_datetime'], format="%m/%d/%y %H:%M")
data_train['dropoff_datetime'] = pd.to_datetime(data_train['dropoff_datetime'], format="%m/%d/%y %H:%M")

In [25]:
data_train

,pickup_datetime,dropoff_datetime,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,passenger_count,traffic_congestion_level,pickup_borough,dropoff_borough,tolls_amount,tip_amount,fare_estimate,trip_distance_miles,lat_diff,lon_diff,euclidean_distance
0,2024-03-22 09:10:00,2024-03-22 09:23:00,40.690045,-74.239699,40.704018,-74.214368,2,high,Other,Other,0.00,0.14,18.40,10.9422,0.013973,0.025331,0.028929
1,2024-08-14 10:48:00,2024-08-14 10:52:00,40.763224,-74.053604,40.759277,-74.066002,1,medium,Other,Other,0.00,1.36,9.99,1.2817,-0.003947,-0.012398,0.013011
2,2024-06-17 03:47:00,2024-06-17 03:55:00,40.658970,-74.179122,40.628631,-74.180385,1,low,Other,Staten Island,5.67,2.36,20.22,13.8034,-0.030339,-0.001263,0.030365
3,2024-06-25 00:45:00,2024-06-25 01:04:00,40.696980,-73.773970,40.670444,-73.838194,2,medium,Brooklyn,Brooklyn,0.00,3.80,29.73,24.2266,-0.026536,-0.064224,0.069490
4,2024-06-10 04:10:00,2024-06-10 04:14:00,40.580638,-73.788101,40.594286,-73.780625,3,low,Brooklyn,Brooklyn,0.00,2.68,11.05,3.4222,0.013648,0.007476,0.015561
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14694,2024-06-21 09:42:00,2024-06-21 09:53:00,40.818300,-74.037218,40.823860,-74.005730,4,medium,Bronx,Bronx,0.00,1.19,14.59,9.4093,0.005560,0.031488,0.031975
14695,2024-09-21 21:39:00,2024-09-21 21:56:00,40.913272,-74.012950,40.936902,-74.031666,1,high,Bronx,Bronx,0.00,3.58,20.40,12.7578,0.023630,-0.018716,0.030144
14696,2024-01-16 23:48:00,2024-01-16 23:57:00,40.808487,-74.072608,40.804565,-74.033420,2,medium,Bronx,Bronx,0.00,2.69,14.64,12.3564,-0.003922,0.039188,0.039384
14697,2024-09-28 09:34:00,2024-09-28 10:27:00,40.605883,-73.902029,40.693266,-73.843018,3,high,Brooklyn,Brooklyn,0.00,13.61,60.69,27.0350,0.087383,0.059011,0.105442


In [26]:
data_train['trip_duration_min'] = (data_train['dropoff_datetime'] - data_train['pickup_datetime']).dt.total_seconds()/ 60
data_train['pickup_hour'] = data_train['pickup_datetime'].dt.hour
data_train['pickup_dayofweek'] = data_train['pickup_datetime'].dt.dayofweek
data_train['pickup_month'] = data_train['pickup_datetime'].dt.month

In [27]:
data_train

,pickup_datetime,dropoff_datetime,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,passenger_count,traffic_congestion_level,pickup_borough,dropoff_borough,...,tip_amount,fare_estimate,trip_distance_miles,lat_diff,lon_diff,euclidean_distance,trip_duration_min,pickup_hour,pickup_dayofweek,pickup_month
0,2024-03-22 09:10:00,2024-03-22 09:23:00,40.690045,-74.239699,40.704018,-74.214368,2,high,Other,Other,...,0.14,18.40,10.9422,0.013973,0.025331,0.028929,13.0,9,4,3
1,2024-08-14 10:48:00,2024-08-14 10:52:00,40.763224,-74.053604,40.759277,-74.066002,1,medium,Other,Other,...,1.36,9.99,1.2817,-0.003947,-0.012398,0.013011,4.0,10,2,8
2,2024-06-17 03:47:00,2024-06-17 03:55:00,40.658970,-74.179122,40.628631,-74.180385,1,low,Other,Staten Island,...,2.36,20.22,13.8034,-0.030339,-0.001263,0.030365,8.0,3,0,6
3,2024-06-25 00:45:00,2024-06-25 01:04:00,40.696980,-73.773970,40.670444,-73.838194,2,medium,Brooklyn,Brooklyn,...,3.80,29.73,24.2266,-0.026536,-0.064224,0.069490,19.0,0,1,6
4,2024-06-10 04:10:00,2024-06-10 04:14:00,40.580638,-73.788101,40.594286,-73.780625,3,low,Brooklyn,Brooklyn,...,2.68,11.05,3.4222,0.013648,0.007476,0.015561,4.0,4,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14694,2024-06-21 09:42:00,2024-06-21 09:53:00,40.818300,-74.037218,40.823860,-74.005730,4,medium,Bronx,Bronx,...,1.19,14.59,9.4093,0.005560,0.031488,0.031975,11.0,9,4,6
14695,2024-09-21 21:39:00,2024-09-21 21:56:00,40.913272,-74.012950,40.936902,-74.031666,1,high,Bronx,Bronx,...,3.58,20.40,12.7578,0.023630,-0.018716,0.030144,17.0,21,5,9
14696,2024-01-16 23:48:00,2024-01-16 23:57:00,40.808487,-74.072608,40.804565,-74.033420,2,medium,Bronx,Bronx,...,2.69,14.64,12.3564,-0.003922,0.039188,0.039384,9.0,23,1,1
14697,2024-09-28 09:34:00,2024-09-28 10:27:00,40.605883,-73.902029,40.693266,-73.843018,3,high,Brooklyn,Brooklyn,...,13.61,60.69,27.0350,0.087383,0.059011,0.105442,53.0,9,5,9


In [28]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14699 entries, 0 to 14698
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   pickup_datetime           14699 non-null  datetime64[ns]
 1   dropoff_datetime          14699 non-null  datetime64[ns]
 2   pickup_latitude           14699 non-null  float64       
 3   pickup_longitude          14699 non-null  float64       
 4   dropoff_latitude          14699 non-null  float64       
 5   dropoff_longitude         14699 non-null  float64       
 6   passenger_count           14699 non-null  int64         
 7   traffic_congestion_level  14699 non-null  object        
 8   pickup_borough            14699 non-null  object        
 9   dropoff_borough           14699 non-null  object        
 10  tolls_amount              14699 non-null  float64       
 11  tip_amount                14699 non-null  float64       
 12  fare_estimate     

In [29]:
num_col=['passenger_count', 'tolls_amount', 'tip_amount', 'trip_duration_min', 'pickup_hour', 'pickup_dayofweek', 'pickup_month', 'lat_diff', 'lon_diff', 'euclidean_distance']
one_hot_col=['pickup_borough', 'dropoff_borough']
ordinal_cols=['traffic_congestion_level']
ordinal_order=['low', 'medium', 'high']
ordinal_cat=[ordinal_order]

In [30]:
x=data_train[num_col+one_hot_col+ordinal_cols]
y=data_train['trip_distance_miles']
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42
)


In [31]:
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
ordinal_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OrdinalEncoder(categories=ordinal_cat, handle_unknown='use_encoded_value', unknown_value=-1))
])

# one-hot + imputation
onehot_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

# numeric imputation
numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ("poly", PolynomialFeatures(degree=2, include_bias=False)),   #### polynomial applies only on numerical features
    ('scaler', StandardScaler())
])

In [32]:
pre_prossesor = ColumnTransformer(
    transformers=[
        ('ordinal_encoder', ordinal_pipeline, ordinal_cols),
        ('onehot_encoder', onehot_pipeline, one_hot_col),
        ('scaler', numeric_pipeline, num_col)
    ]
)

In [33]:
model = Pipeline(steps=[
    ("preprocess", pre_prossesor),
    ("regressor", LinearRegression())
])

model.fit(x_train, y_train)
y_pred = model.predict(x_test)

In [34]:
y_test_predict=model.predict(x_test)
y_train_predict=model.predict(x_train)

In [35]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# --- Train ---
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_predict))
train_mae  = mean_absolute_error(y_train, y_train_predict)
train_r2   = r2_score(y_train, y_train_predict)

# --- Test ---
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_predict))
test_mae  = mean_absolute_error(y_test, y_test_predict)
test_r2   = r2_score(y_test, y_test_predict)

print("\n===== TRAIN METRICS =====")
print("RMSE :", train_rmse)
print("MAE  :", train_mae)
print("R²   :", train_r2)

print("\n===== TEST METRICS =====")
print("RMSE :", test_rmse)
print("MAE  :", test_mae)
print("R²   :", test_r2)


===== TRAIN METRICS =====
RMSE : 2.2783731381400374
MAE  : 1.744755444300931
R²   : 0.9165601121374227

===== TEST METRICS =====
RMSE : 2.2916567858336063
MAE  : 1.717779229015281
R²   : 0.9168683602755964


In [36]:
data_test.isna().sum().sort_values(ascending=False)

,0
trip_distance_miles,300
pickup_datetime,0
pickup_latitude,0
pickup_longitude,0
dropoff_latitude,0
dropoff_datetime,0
dropoff_longitude,0
passenger_count,0
pickup_borough,0
traffic_congestion_level,0


In [38]:
data_test['lat_diff'] = data_test['dropoff_latitude'] - data_test['pickup_latitude']
data_test['lon_diff'] = data_test['dropoff_longitude'] - data_test['pickup_longitude']
data_test['euclidean_distance'] = np.sqrt(data_test['lat_diff']**2 + data_test['lon_diff']**2)

In [39]:
data_test['pickup_datetime'] = pd.to_datetime(data_test['pickup_datetime'], format="%m/%d/%y %H:%M")
data_test['dropoff_datetime'] = pd.to_datetime(data_test['dropoff_datetime'], format="%m/%d/%y %H:%M")


In [40]:
data_test['trip_duration_min'] = (data_test['dropoff_datetime'] - data_test['pickup_datetime']).dt.total_seconds()/ 60
data_test['pickup_hour'] = data_test['pickup_datetime'].dt.hour
data_test['pickup_dayofweek'] = data_test['pickup_datetime'].dt.dayofweek
data_test['pickup_month'] = data_test['pickup_datetime'].dt.month

In [41]:
num_col=['passenger_count', 'tolls_amount', 'tip_amount', 'trip_duration_min', 'pickup_hour', 'pickup_dayofweek', 'pickup_month', 'lat_diff', 'lon_diff', 'euclidean_distance']
one_hot_col=['pickup_borough', 'dropoff_borough']
ordinal_cols=['traffic_congestion_level']
ordinal_order=['low', 'medium', 'high']
ordinal_cat=[ordinal_order]

In [43]:
x=data_test[num_col+one_hot_col+ordinal_cols]
y=data_test['trip_distance_miles']
y_test_predict=model.predict(x)

In [47]:
#Ridge
from sklearn.linear_model import Ridge, Lasso, ElasticNet

Ridge_model = Pipeline(steps=[
    ("preprocess", pre_prossesor),
    ("ridge", Ridge(alpha=1.0))
])
Ridge_model.fit(x_train, y_train)
y_test_predict_Ridge=Ridge_model.predict(x_test)
y_train_predict_Ridge=Ridge_model.predict(x_train)

In [49]:
#Lasso
Lasso_model = Pipeline(steps=[
    ("preprocess", pre_prossesor),
    ("lasso", Lasso(alpha=1.0))
])
Lasso_model.fit(x_train, y_train)
y_test_predict_Lasso=Lasso_model.predict(x_test)
y_train_predict_Lasso=Lasso_model.predict(x_train)


In [50]:
#Elastic
Elastic_model = Pipeline(steps=[
    ("preprocess", pre_prossesor),
    ("elastic", ElasticNet(alpha=1.0, l1_ratio=0.5))
])
Elastic_model.fit(x_train, y_train)
y_test_predict_Elastic=Elastic_model.predict(x_test)
y_train_predict_Elastic=Elastic_model.predict(x_train)

In [52]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# --- Lasso_Train ---
train_rmse_Lasso = np.sqrt(mean_squared_error(y_train, y_train_predict_Lasso))
train_mae_Lasso  = mean_absolute_error(y_train, y_train_predict_Lasso)
train_r2_Lasso  = r2_score(y_train, y_train_predict_Lasso)

# --- Lasso Test ---
test_rmse_Lasso = np.sqrt(mean_squared_error(y_test, y_test_predict_Lasso))
test_mae_Lasso  = mean_absolute_error(y_test, y_test_predict_Lasso)
test_r2_Lasso   = r2_score(y_test, y_test_predict_Lasso)

print("\n===== TRAIN METRICS Lasso =====")
print("RMSE :", train_rmse_Lasso)
print("MAE  :", train_mae_Lasso)
print("R²   :", train_r2_Lasso)

print("\n===== TEST METRICS Lasso =====")
print("RMSE :", test_rmse_Lasso)
print("MAE  :", test_mae_Lasso)
print("R²   :", test_r2_Lasso)


===== TRAIN METRICS Lasso =====
RMSE : 4.323906188991986
MAE  : 3.606500350771426
R²   : 0.699477817390562

===== TEST METRICS Lasso =====
RMSE : 4.286175244747368
MAE  : 3.6497921870390067
R²   : 0.709191535301243


In [53]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# --- Ridge_Train ---
train_rmse_Ridge = np.sqrt(mean_squared_error(y_train, y_train_predict_Ridge))
train_mae_Ridge  = mean_absolute_error(y_train, y_train_predict_Ridge)
train_r2_Ridge   = r2_score(y_train, y_train_predict_Ridge)

# --- Ridge Test ---
test_rmse_Ridge = np.sqrt(mean_squared_error(y_test, y_test_predict_Ridge))
test_mae_Ridge  = mean_absolute_error(y_test, y_test_predict_Ridge)
test_r2_Ridge   = r2_score(y_test, y_test_predict_Ridge)

print("\n===== TRAIN METRICS Ridge =====")
print("RMSE :", train_rmse_Ridge)
print("MAE  :", train_mae_Ridge)
print("R²   :", train_r2_Ridge)

print("\n===== TEST METRICS Ridge =====")
print("RMSE :", test_rmse_Ridge)
print("MAE  :", test_mae_Ridge)
print("R²   :", test_r2_Ridge)


===== TRAIN METRICS Ridge =====
RMSE : 2.2783922370251637
MAE  : 1.7453617037856297
R²   : 0.9165587132309083

===== TEST METRICS Ridge =====
RMSE : 2.2902963345892258
MAE  : 1.7181366456875256
R²   : 0.9169670338407666


In [55]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# --- Elastic_Train ---
train_rmse_Elastic = np.sqrt(mean_squared_error(y_train, y_train_predict_Elastic))
train_mae_Elastic  = mean_absolute_error(y_train, y_train_predict_Elastic)
train_r2_Elastic   = r2_score(y_train, y_train_predict_Elastic)

# --- Elastic Test ---
test_rmse_Elastic = np.sqrt(mean_squared_error(y_test, y_test_predict_Elastic))
test_mae_Elastic  = mean_absolute_error(y_test, y_test_predict_Elastic)
test_r2_Elastic   = r2_score(y_test, y_test_predict_Elastic)

print("\n===== TRAIN METRICS Elastic =====")
print("RMSE :", train_rmse_Elastic)
print("MAE  :", train_mae_Elastic)
print("R²   :", train_r2_Elastic)

print("\n===== TEST METRICS Elastic =====")
print("RMSE :", test_rmse_Elastic)
print("MAE  :", test_mae_Elastic)
print("R²   :", test_r2_Elastic)


===== TRAIN METRICS Elastic =====
RMSE : 4.585187613791583
MAE  : 3.8610717642073817
R²   : 0.6620610662328392

===== TEST METRICS Elastic =====
RMSE : 4.557458113953028
MAE  : 3.9145522355547753
R²   : 0.6712145666956882
